# Serverless

In [6]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.applications.xception import preprocess_input

In [2]:
!wget https://github.com/alexeygrigorev/mlbookcamp-code/releases/download/chapter7-model/xception_v4_large_08_0.894.h5 -O clothing_model.h5 --quiet

In [4]:
model = keras.models.load_model('clothing_model.h5')

In [5]:
!wget http://bit.ly/mlbookcamp-pants -O pants.jpg --quiet

In [7]:
img = load_img('pants.jpg', target_size=(299, 299))

In [8]:
x = np.array(img)
X = np.array([x])
X = preprocess_input(X)

In [9]:
X.shape

(1, 299, 299, 3)

In [10]:
preds = model.predict(X)
preds

1/1 [==============================] - 1s 1s/step


array([[-1.8682904, -4.7612453, -2.3169832, -1.0625703,  9.887159 ,
        -2.8124344, -3.666283 ,  3.2003617, -2.602338 , -4.835046 ]],
      dtype=float32)

In [11]:
labels = [
    'dress',
    'hat',
    'longsleeve',
    'outwear',
    'pants',
    'shirt',
    'shoes',
    'shorts',
    'skirt',
    't-shirt'
]

results = dict(zip(labels, preds[0]))
results

{'dress': -1.8682904,
 'hat': -4.7612453,
 'longsleeve': -2.3169832,
 'outwear': -1.0625703,
 'pants': 9.887159,
 'shirt': -2.8124344,
 'shoes': -3.666283,
 'shorts': 3.2003617,
 'skirt': -2.602338,
 't-shirt': -4.835046}

# Convert Keras to TF-Lite

In [12]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)

tflite_model = converter.convert()

with tf.io.gfile.GFile('clothing-model.tflite', 'wb') as f:
    f.write(tflite_model)

In [13]:
import tensorflow.lite as tflite

In [14]:
interpreter = tflite.Interpreter(model_path='clothing-model.tflite')
interpreter.allocate_tensors()

In [15]:
interpreter.get_input_details()

[{'name': 'serving_default_input_8:0',
  'index': 0,
  'shape': array([  1, 299, 299,   3], dtype=int32),
  'shape_signature': array([ -1, 299, 299,   3], dtype=int32),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

In [16]:
input_index = interpreter.get_input_details()[0]['index']

In [21]:
interpreter.get_output_details()

[{'name': 'StatefulPartitionedCall:0',
  'index': 229,
  'shape': array([ 1, 10], dtype=int32),
  'shape_signature': array([-1, 10], dtype=int32),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

In [22]:
output_index = interpreter.get_output_details()[0]['index']

In [23]:
interpreter.set_tensor(input_index, X)

In [24]:
interpreter.invoke()

In [25]:
preds = interpreter.get_tensor(output_index)

array([[-1.8682916, -4.7612457, -2.316979 , -1.0625672,  9.8871565,
        -2.8124275, -3.666287 ,  3.2003636, -2.6023414, -4.8350444]],
      dtype=float32)

In [26]:
labels = [
    'dress',
    'hat',
    'longsleeve',
    'outwear',
    'pants',
    'shirt',
    'shoes',
    'shorts',
    'skirt',
    't-shirt'
]

results = dict(zip(labels, preds[0]))
results

{'dress': -1.8682904,
 'hat': -4.7612453,
 'longsleeve': -2.3169832,
 'outwear': -1.0625703,
 'pants': 9.887159,
 'shirt': -2.8124344,
 'shoes': -3.666283,
 'shorts': 3.2003617,
 'skirt': -2.602338,
 't-shirt': -4.835046}